#

# Series, or Open? How to maximize your rank points

## Imports and stuff

In [ ]:
import random
import numpy as np

## Constants
Let's start by defining all the relevant constants, which mostly depend on rank. I'm gonna use dictionaries here, with the rank names as keys.

In [ ]:
# Open
POINTS_PER_OPEN_WIN = {
    "C-": 20,
    "C": 20,
    "C+": 20,
    "B-": 20,
    "B": 20,
    "B+": 20,
    "A-": 20,
    "A": 20,
    "A+": 20,
    "S": 20,
    "S+": 20,
}
POINTS_PER_OPEN_LOSS = {
    "C-": -3,
    "C": -3,
    "C+": -3,
    "B-": -5,
    "B": -5,
    "B+": -5,
    "A-": -8,
    "A": -8,
    "A+": -8,
    "S": -10,
    "S+": -13,
}

# Series
ENTRY_FEE = {
    "C-": 0,
    "C": -20,
    "C+": -40,
    "B-": -55,
    "B": -70,
    "B+": -85,
    "A-": -110,
    "A": -120,
    "A+": -130,
    "S": -170,
    "S+": -180,
}
SERIES_WIN_POINTS = {
    "C-": 20,
    "C": 20,
    "C+": 20,
    "B-": 30, 
    "B": 30,
    "B+": 30,
    "A-": 40,
    "A": 40,
    "A+": 40,
    "S": 50,
    "S+": 50,
}


In [ ]:
def calculate_series_win_points(rank: str, wins: int) -> int:
    points_per_win = SERIES_WIN_POINTS[rank]
    total_points = 0
    for i in range(wins):
        total_points += points_per_win + (i * 5)
    return total_points

In [ ]:
def simulate_series(rank: str, win_ratio: float, gold_medals: int, silver_medals: int) -> tuple[int, int, int]:
    points = ENTRY_FEE[rank]
    
    # Calculate medal points
    assert gold_medals + silver_medals <= 3, "Total medals cannot exceed 3"
    medal_points = gold_medals * 5 + silver_medals * 1

    # Simulate until 5 wins or 3 losses
    wins = 0
    losses = 0
    while wins < 5 and losses < 3:
        if random.random() < win_ratio:
            # Win
            points += SERIES_WIN_POINTS[rank] + (wins * 5) + medal_points
            wins += 1
        else:
            # Loss
            losses += 1

    return (points, wins, losses) 

In [ ]:
def simulate_multiple_series(rank: str, win_ratio: float, gold_medals: int, silver_medals: int, epsilon: float = 1e-5, min_matches: int=1000) -> tuple[float, float, float, int]:
    total_points = 0
    total_wins = 0
    total_losses = 0

    average = 1
    new_average = -1

    while abs(average - new_average) > epsilon or (total_wins + total_losses) < min_matches:
        average = new_average

        points, wins, losses = simulate_series(rank, win_ratio, gold_medals, silver_medals)
        total_points += points
        total_wins += wins
        total_losses += losses
        
        new_average = total_points / (total_wins + total_losses + 1e-9)

    average_points = total_points / (total_wins + total_losses)
    average_wins = total_wins / (total_wins + total_losses)
    average_losses = total_losses / (total_wins + total_losses)

    return (average_points, average_wins, average_losses, total_wins + total_losses)

In [ ]:
rank = "S+"
win_ratio = 0.4
average_points, average_wins, average_losses, total_matches = simulate_multiple_series(
    rank,
    win_ratio,
    gold_medals=3,
    silver_medals=0,
)

print(f"Rank: {rank}, Win Ratio: {win_ratio:.2f}")
print()
print(f"### Series ###")
print(f"Average Points per match: {average_points:.2f}")
print()
print(f"### Open ###")
print(f"Average Points per match: {POINTS_PER_OPEN_WIN[rank] * win_ratio + POINTS_PER_OPEN_LOSS[rank] * (1 - win_ratio):.2f}")

In [ ]:
win_ratios = [i / 100 for i in range(0, 101)]
# medal_combinations = [(3, 0), (2, 1), (1, 2), (0, 3)] + [(2, 0), (1, 1), (0, 2)] + [(1, 0), (0, 1)] + [(0, 0)]
medal_combinations = [(3, 0), (2, 0), (1, 0), (0, 0)]

open_points = [POINTS_PER_OPEN_WIN[rank] * wr + POINTS_PER_OPEN_LOSS[rank] * (1 - wr) for wr in win_ratios]


# Visualization

In [ ]:
import matplotlib.pyplot as plt

# Make plots bigger
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:

# Plot a heatmap for series points
for gold_medals, silver_medals in medal_combinations:
    average_points_list = []
    for wr in win_ratios:
        avg_points, _, _, _ = simulate_multiple_series(
            rank,
            wr,
            gold_medals,
            silver_medals,
            epsilon=1e-4,
        )
        average_points_list.append(avg_points)
        
    plt.plot(win_ratios, average_points_list, label=f"Series Points (Gold: {gold_medals}, Silver: {silver_medals})")

# Plot a horizontal red line at y=0
plt.axhline(0, color='red', linestyle='--', linewidth=1)

# Plot the points per match for open
plt.plot(win_ratios, open_points, label="Open Points per Match", linewidth=2)

# Labels and stuff
# plt.ylim(min(open_points), max(open_points))
plt.xlim(0, 1)
plt.xlabel("Win Ratio")
plt.xticks([i / 10 for i in range(0, 11)], [f"{i*10}%" for i in range(0, 11)])
plt.ylabel("Points per Match")
plt.title(f"Points per Match vs Win Ratio for Rank {rank}")
plt.legend()
plt.grid()
plt.show()

This is interesting, particularly how as the win ratio increases there seems to be diminishing returns.

My theory is that this is because medals in losses still give you points, which means that the more games the better. If you are winning a lot, you are just playing paying the entry fee for 5 games.

This doesn't include **throwing**, big TODO!